# Spark practical work
### Melen Laclais, Carlos Manzano Izquierdo

## Load of training data 

In [ ]:

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, when, isnan

# Initialize Spark Session
spark = SparkSession.builder \
    .appName("Flight_Delay_EDA_Analysis") \
    .config("spark.driver.memory", "8g") \
    .master("local[*]") \
    .getOrCreate()

# Load Training Data (2006 + 2007)
print("Loading raw flight data (2006-2007)... This may take a moment due to schema inference...")

# We use inferSchema=True to see the actual data types Spark detects
# We use nullValue="NA" because the documentation states "NA" is used for nulls
flights_raw = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("nullValue", "NA") \
    .csv(["../training_data/flight_data/2006.csv.bz2", "../training_data/flight_data/2007.csv.bz2"])

# # Load auxiliary tables for inspection
# planes_df = spark.read.option("header", "true").option("inferSchema", "true").option("nullValue", "NA").csv("../training_data/flight_data/plane-data.csv")
# airports_df = spark.read.option("header", "true").option("inferSchema", "true").option("nullValue", "NA").csv("../training_data/flight_data/airports.csv")

print(f"Total flights loaded: {flights_raw.count():,}")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/12/27 23:14:45 WARN Utils: Your hostname, MSICarlos, resolves to a loopback address: 127.0.1.1; using 192.168.1.182 instead (on interface wlo1)
25/12/27 23:14:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/27 23:14:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Loading raw flight data (2006-2007)... This may take a moment due to schema inference...


Total flights loaded: 14,595,137


## Dataset exploration, analysis and processing

First, lets start by analyzing the schema of the dataset

In [ ]:

print("--- Inferred Schema ---")
flights_raw.printSchema()

#(using Pandas for better formatting in Jupyter)
print("--- Data Sample (First 5 rows) ---")
flights_raw.limit(5).toPandas()

--- Inferred Schema ---
root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: integer (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- ArrTime: integer (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: integer (nullable = true)
 |-- CRSElapsedTime: integer (nullable = true)
 |-- AirTime: integer (nullable = true)
 |-- ArrDelay: integer (nullable = true)
 |-- DepDelay: integer (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: integer (nullable = true)
 |-- TaxiIn: integer (nullable = true)
 |-- TaxiOut: integer (nullable = true)
 |-- Cancelled: integer (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: i

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2006,1,11,3,743,745,1024,1018,US,343,...,45,13,0,None,0,0,0,0,0,0
1,2006,1,11,3,1053,1053,1313,1318,US,613,...,27,19,0,None,0,0,0,0,0,0
2,2006,1,11,3,1915,1915,2110,2133,US,617,...,4,11,0,None,0,0,0,0,0,0
3,2006,1,11,3,1753,1755,1925,1933,US,300,...,16,10,0,None,0,0,0,0,0,0
4,2006,1,11,3,824,832,1015,1015,US,765,...,27,12,0,None,0,0,0,0,0,0



### Delete Forbidden variables


Now the first step that we will take is to delete all of the variables whose information would not be available during inference, the "Forbidden" variables:

* ArrTime
* ActualElapsedTime
* AirTime
* TaxiIn
* Diverted
* CarrierDelay
* WeatherDelay
* NASDelay
* SecurityDelay
* LateAircraftDelay

In [13]:

# These variables contain future information known only after landing.
forbidden_vars = [
    "ArrTime",
    "ActualElapsedTime",
    "AirTime",
    "TaxiIn",
    "Diverted",
    "CarrierDelay",
    "WeatherDelay",
    "NASDelay",
    "SecurityDelay",
    "LateAircraftDelay"
]

# Drop the forbidden columns
flights_clean = flights_raw.drop(*forbidden_vars)

print("--- Structure after removing forbidden variables ---")
flights_clean.printSchema()

print("Ensure these variables are not anymore present:\n")

error_found = False
for var in forbidden_vars:
    if var in flights_clean.columns:
        print(f"ERROR: {var} is still present!")
        error_found = True
        
if error_found:
    print("Some forbidden variables are still present. Please check the code.")
else:
    print("All forbidden variables successfully removed.")


--- Structure after removing forbidden variables ---
root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: integer (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- CRSElapsedTime: integer (nullable = true)
 |-- ArrDelay: integer (nullable = true)
 |-- DepDelay: integer (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: integer (nullable = true)
 |-- TaxiOut: integer (nullable = true)
 |-- Cancelled: integer (nullable = true)
 |-- CancellationCode: string (nullable = true)

Ensure these variables are not anymore present:

All forbidden variables successfully removed.



### Delete canceled flights and associated columns

The first cleaning step we must do is removing all the rows whose flights were cancelled, and delete the columns which express if the flight were canceled, as they won't have useful information anymore.

In [ ]:
flights_clean = flights_clean.filter("Cancelled == 0") \
                            .drop("Cancelled", "CancellationCode")
                            
flights_clean.printSchema()                            
print(f"Total valid flights for training: {flights_clean.count():,}")


root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: integer (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- CRSElapsedTime: integer (nullable = true)
 |-- ArrDelay: integer (nullable = true)
 |-- DepDelay: integer (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: integer (nullable = true)
 |-- TaxiOut: integer (nullable = true)



Total valid flights for training: 14,312,455


### Data quality analysis
Now we will keep assesing the quality of the features that we have available, to verify which techniques of imputation and deletion we should follow. We will also evaluate if we need to transform into categorical some of the variables,  and the consistence showed in their values. 


In [2]:
print("--- Missing Values Analysis ---")
# Calculate count of nulls for each column
null_counts = flights_clean.select([count(when(col(c).isNull(), c)).alias(c) for c in flights_raw.columns])
null_counts.show(truncate=False)

--- Missing Values Analysis ---


NameError: name 'flights_clean' is not defined

### Cleaning and application of transformations
(*Smart handling of special format in some input variables, performing relevant processing
and/or transformations.*)

### Join dataset with other files to obtain useful information
(*exploring additional datasets to try to find additional relevant
information*)

### Agreggation and mixing of columns in order to create new columns that could be useful 

### EDA and statistical analysis over these features
*(Proper exploratory data analysis (possibly including univariate and/or multivariate
analysis) to better understand the input data and provide robust criteria for variable
selection.)*

### Feature Engineering

(*Feature engineering*) 

### PCA ???

## Code used to train, test and save the model.

### Scaling of values ? 

### Definition of models to employ and metrics to use 
* *Select more than one valid machine learning algorithm for building the model.*
* *Consider more than one possible model performance metric and explain the criteria for
selecting the most appropriate*

### Cross-Validation loop to validate models with 80/20
* *Use cross-validation techniques to select the best model*
* *Perform model hyper-parameter tuning*
* *Use the full capacities of Spark’s MLlib*


### Selection and saving the best model 